In [5]:
%pip install geopandas
%pip install pyrosm
%pip install gtfs_kit
%pip install h3
%pip install keplergl
%pip install matplotlib
%pip install esy-osmfilter

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 1.0 MB 814 kB/s eta 0:00:01     |██████████████████              | 573 kB 814 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sys
from pathlib import Path
import json
import h3

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

import gtfs_kit as gk

from shapely.geometry import mapping, shape

In [2]:
path = "../resources/rejseplanen.zip"
feed = gk.read_feed(path, dist_units='km')

dates = ['20220320', '20220321', '20220322', '20220323', '20220324', '20220325', '20220326']

In [3]:
stops = feed.stops

stops['h3'] = stops.apply(lambda row: h3.geo_to_h3(row['stop_lat'],row['stop_lon'],10), axis=1)
stops.head()

stop_h3_map = pd.Series(stops.h3.values,index=stops.stop_id).to_dict()

In [ ]:
geojson = feed.routes_to_geojson(include_stops=True)

In [8]:
with open("geojson.json", "w") as outfile:
    json.dump(geojson, outfile)

In [4]:
ts = feed.compute_stop_time_series(dates=dates, freq='60Min')
ts.shape

(168, 38276)

In [5]:
tts = ts.transpose().reset_index(level=1)
tts.head()

datetime,stop_id,2022-03-20 00:00:00,2022-03-20 01:00:00,2022-03-20 02:00:00,2022-03-20 03:00:00,2022-03-20 04:00:00,2022-03-20 05:00:00,2022-03-20 06:00:00,2022-03-20 07:00:00,2022-03-20 08:00:00,...,2022-03-26 14:00:00,2022-03-26 15:00:00,2022-03-26 16:00:00,2022-03-26 17:00:00,2022-03-26 18:00:00,2022-03-26 19:00:00,2022-03-26 20:00:00,2022-03-26 21:00:00,2022-03-26 22:00:00,2022-03-26 23:00:00
indicator,,,,,,,,,,,,,,,,,,,,,
num_trips,000000000006,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,...,6.0,6.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0
num_trips,000000000008,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,...,6.0,6.0,5.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0
num_trips,000000000009,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,...,6.0,6.0,5.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0
num_trips,000000000011,3.0,1.0,1.0,1.0,1.0,2.0,4.0,8.0,8.0,...,17.0,17.0,16.0,10.0,9.0,9.0,8.0,9.0,8.0,8.0
num_trips,000000000012,2.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,...,9.0,9.0,8.0,6.0,6.0,6.0,5.0,6.0,5.0,5.0


In [6]:
def add_lists(lists):
    res = np.array([0]*len(lists[0]))
    for i in range(len(lists)-1):
        l = np.array(lists[i])
        l = np.nan_to_num(l)
        # print(f"adding res with {np.shape(res)} and list with {np.shape(l)}")
        res = np.add(res, l)
        # print(f"resulting shape: {np.shape(res)}")
    return res

In [7]:
result_df = pd.DataFrame()
# result_df['h3'] = tts.apply(lambda row: stop_h3_map[row['stop_id']], axis=1)
result_df['stop_id'] = tts['stop_id'].values.tolist()
result_df['h3'] = result_df.apply(lambda row: stop_h3_map[row['stop_id']], axis=1)
result_df['frequency'] = tts.iloc[:,1:].values.tolist()

unique_h3s = set(stop_h3_map.values())
h3_map = {}
for index in unique_h3s:
    res = result_df.loc[result_df['h3'] == index]
    if res.shape[0] > 1:
        res = add_lists(res['frequency'].values.tolist())
        h3_map[index] = res.tolist()
    elif res['frequency'].any():
        h3_map[index] = np.nan_to_num(res['frequency'].values[0]).tolist()
    

In [8]:
new = pd.DataFrame()
new['h3'] = h3_map.keys()
new['freq'] = h3_map.values()
new.head()

,h3,freq
0,8a1f0425e1a7fff,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,8a1f230dc557fff,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,8a1f230432e7fff,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,8a1f26030607fff,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,8a1f246d6097fff,"[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 2.0, ..."


In [ ]:
new.to_csv("data.csv")

In [ ]:
time_cols = tts.columns[1:]

In [ ]:
insane_array = []
for key, value in h3_map.items():
    for i, v in enumerate(value):
        insane_array.append([key, str(time_cols[i]), v])

insane_df = pd.DataFrame(insane_array, columns=['hex_id', 'time', 'height'])

In [ ]:
from keplergl import KeplerGl
map = KeplerGl(height=500, data={'data': insane_df})
map.save_to_html()

### Read in data from osm_convert.py

In [11]:
import json
with open('dkall.geojson') as json_file:
    data = json.load(json_file)


print(data['features'][0])
for geo in data['features']:
    geo['geometry']['coordinates'] =  [[y,x] for x,y in geo['geometry']['coordinates']]

print(data['features'][0])


{'type': 'Feature', 'OSM-id': 2081, 'geometry': {'type': 'LineString', 'coordinates': [[12.068616900000002, 55.63906129999988], [12.068575599999985, 55.63924110000002], [12.068572399999985, 55.63925310000002], [12.068513299999985, 55.63947020000002], [12.068505499999985, 55.63949900000002], [12.068481799999985, 55.63958620000002], [12.068464999999986, 55.639647900000014], [12.068336000000018, 55.64012210000009], [12.068039499999996, 55.64127480000004], [12.0680057, 55.64140629999993], [12.067493399999968, 55.64315839999993], [12.067267600000006, 55.64391739999991], [12.067134999999988, 55.64428009999988], [12.067132999999988, 55.644330099999884], [12.067133699999989, 55.64436989999988], [12.06709269999999, 55.644525199999876], [12.067073900000047, 55.64459120000007], [12.0669844, 55.64488570000034], [12.066940200000007, 55.645036699999906], [12.06662590000002, 55.64611490000006], [12.066515300000045, 55.646507399999734], [12.066376499999956, 55.64699540000023], [12.066336100000006, 55.

# Transform

In [126]:
from pyproj import Transformer, CRS


crs_4326 = CRS.from_epsg(4326)
crs_25831 = CRS.from_epsg(25832)

to_dk = Transformer.from_crs(crs_4326, crs_25831)
to_WGS = Transformer.from_crs(crs_25831, crs_4326)


def CRS_transform(shp, inv=False):
    geoInterface = shp.__geo_interface__

    shpType = geoInterface['type']
    coords = geoInterface['coordinates']
    if shpType == 'Polygon':
        if inv:
            newCoord = [[to_WGS.transform(*point) for point in linring] for linring in coords]
        else:
            newCoord = [[to_dk.transform(*point) for point in linring] for linring in coords]
    elif shpType == 'LineString':
        if inv:
            newCoord = [to_WGS.transform(*point) for point in coords]
        else:
            newCoord = [to_dk.transform(*point) for point in coords]
    elif shpType == 'MultiPolygon':
        if inv:
            newCoord = [[[to_WGS.transform(*point) for point in linring] for linring in poly] for poly in coords]
        else:
            newCoord = [[[to_dk.transform(*point) for point in linring] for linring in poly] for poly in coords]

    return shape({'type': shpType, 'coordinates': tuple(newCoord)})

# Polyfill method

In [ ]:
from shapely.geometry import mapping, shape

#consider how buffer works for WSG82

h3_set = set()
for geo in data['features']:
    tmp = shape(geo['geometry'])
    tmp = CRS_transform(tmp)
    tmp = tmp.buffer(10)
    tmp = CRS_transform(tmp, inv = True)

    if mapping(tmp)['type'] == 'Polygon':
        h3_set.update(h3.polyfill(mapping(tmp), 11, geo_json_conformant=False))
    elif mapping(tmp)['type'] == 'MultiPolygon':
        polygons = list(tmp)
        for poly in polygons:
            h3_set.update(h3.polyfill(mapping(poly), 11, geo_json_conformant=False))
    else:
        break
    

In [129]:
walk = pd.DataFrame(list(h3_set), columns=['h3'])
walk['x'] = 1
walk.to_csv("walk_paths_15m.csv")

# H3 line method

In [12]:

h3_set = set()
for geo in data['features']:
    tmp = shape(geo['geometry'])
    if len(mapping(tmp)['coordinates']) == 1:
        h3_set.add(h3.geo_to_h3(*mapping(tmp)['coordinates'], resolution = 11))
    for i in range(len(mapping(tmp)['coordinates']) - 1):
        h3s = h3.h3_line( h3.geo_to_h3(*mapping(tmp)['coordinates'][i], resolution = 11) , h3.geo_to_h3(*mapping(tmp)['coordinates'][i+1], resolution = 11) )
        h3_set.update(h3s)

In [ ]:
walk = pd.DataFrame(list(h3_set), columns=['h3'])
walk.to_csv("line_paths_11.csv")

# H3 edges method

In [175]:
h3_set = set()
for geo in data['features']:
    tmp = shape(geo['geometry'])
    if len(mapping(tmp)['coordinates']) == 1:
        continue
        h3_set.add(h3.geo_to_h3(*mapping(tmp)['coordinates'], resolution = 11))
    for i in range(len(mapping(tmp)['coordinates']) - 1):
        h3s = h3.h3_line( h3.geo_to_h3(*mapping(tmp)['coordinates'][i], resolution = 11) , h3.geo_to_h3(*mapping(tmp)['coordinates'][i+1], resolution = 11) )
        h3_edges = [h3.get_h3_unidirectional_edge(h3s[i], h3s[i+1]) for i in range(len(h3s) -1)]
        h3_set.update(h3_edges)


In [ ]:
walk = pd.DataFrame(list(h3_set), columns=['h3'])
walk.to_csv("edges_paths_11.csv")

# Line interpolation

In [153]:
from shapely.geometry import mapping, shape, LineString
import numpy as np


h3_set = set()
for geo in data['features']:
    tmp = shape(geo['geometry'])
    tmp = CRS_transform(tmp)
    distances = np.arange(0, tmp.length, 10)
    points = [tmp.interpolate(distance) for distance in distances] 
    if len(points) == 1:
        continue
    tmp = LineString(points)
    tmp = CRS_transform(tmp, inv = True)
    h3s = {h3.geo_to_h3(lat, lon, 11) for (lat, lon) in mapping(tmp)['coordinates']}
    h3_set.update(h3s)
    

KeyboardInterrupt: 

In [161]:
walk = pd.DataFrame(list(h3_set), columns=['h3'])
walk['x'] = 1
walk.to_csv("line_paths_11.csv")